In [43]:
#importar a lib findspark
import findspark
findspark.init()

In [44]:
# importação do PySpark
from pyspark import SparkContext
spark_contexto = SparkContext.getOrCreate()
print(spark_contexto)
print(spark_contexto.version)

<SparkContext master=local[*] appName=pyspark-shell>
3.5.2


In [45]:
# importar o SparkSession a partir do pyspark.sql
from pyspark.sql import SparkSession
#criar seção do Spark
spark = SparkSession.builder.getOrCreate()
print(spark)

In [46]:
#criar dataframe clientes
df_clientes = spark.read.csv('.data/clientes.csv', header=True, inferSchema=True)
df_clientes.printSchema() # visualizar a estrutura desses dados
df_clientes.head() # cabeçalho das colunas e o conteúdo da primeira linha

root
 |-- Cliente: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Endereço: string (nullable = true)
 |-- Contato: string (nullable = true)
 |-- Ordem de Serviço: integer (nullable = true)



Row(Cliente='Leite', Tipo='Restaurante', Cidade='Recife', Endereço='Praça Joaquim Nabuco, 147 - Santo Antônio', Contato='(81) 3224-7977', Ordem de Serviço=2287)

In [47]:
df_pagamentos = spark.read.csv('.data/pagamentos.csv', header=True, inferSchema=True)
df_pagamentos.printSchema()
df_equipamentos = spark.read.csv('.data/equipamentos.csv', header=True, inferSchema=True)
df_equipamentos.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Cliente: string (nullable = true)
 |-- Pagamento : string (nullable = true)
 |-- Forma: string (nullable = true)

root
 |-- Cliente: string (nullable = true)
 |-- Equipamento: string (nullable = true)
 |-- Data do Reparo: string (nullable = true)
 |-- Observações: string (nullable = true)



In [48]:
#união de dfs
df_joined = df_clientes.join(df_pagamentos, 'Cliente', 'inner')
dataset = df_joined # Atribuir a união a dataset
dataset.head()

Row(Cliente='Leite', Tipo='Restaurante', Cidade='Recife', Endereço='Praça Joaquim Nabuco, 147 - Santo Antônio', Contato='(81) 3224-7977', Ordem de Serviço=2287, ID=1, Pagamento ='Pago', Forma=' Á vista')

In [49]:
#mover coluna ID do dataset  para o inicio do dataframe
dataset = dataset.select('ID', *[col for col in dataset.columns if col != 'ID'])
dataset.head()

Row(ID=1, Cliente='Leite', Tipo='Restaurante', Cidade='Recife', Endereço='Praça Joaquim Nabuco, 147 - Santo Antônio', Contato='(81) 3224-7977', Ordem de Serviço=2287, Pagamento ='Pago', Forma=' Á vista')

In [50]:
# Inserindo ID que esta dataset para df_equipamentos
from pyspark.sql.functions import monotonically_increasing_id

# Adiciona uma coluna de ID temporária ao df_equipamentos para ajudar na junção
df_equipamentos_com_id = df_equipamentos.withColumn("temp_id", monotonically_increasing_id())

# Realiza a junção com base na coluna 'cliente'
df_equipamentos_com_id = df_equipamentos_com_id.join(dataset.select("ID", "cliente"), "cliente", "left")

# Seleciona as colunas desejadas, incluindo a nova coluna 'ID'
df_equipamentos_com_id = df_equipamentos_com_id.select("ID", *[col for col in df_equipamentos.columns])

# 
df_equipamentos = df_equipamentos_com_id.drop("temp_id")
df_equipamentos.show()

+---+--------------------+--------------------+--------------+--------------------+
| ID|             Cliente|         Equipamento|Data do Reparo|         Observações|
+---+--------------------+--------------------+--------------+--------------------+
|  5|           Beijupirá|    Fogão Industrial|    18/01/2024|Substituição de q...|
|  7|             Ponteio|               Coifa|    19/01/2024|    Manutenção geral|
| 16|    Oficina do Sabor|    Forno Industrial|    01/02/2024| Troca de termostato|
|  4| Spettus Steak House| Caldeira Industrial|    21/03/2024|    Limpeza e ajuste|
| 10|Bar Restaurante S...|Liquidificador In...|    25/01/2024|Verificação de motor|
| 22|      Empório Recife|       Chapa Bifeira|    15/03/2024|Substituição de p...|
| 17|    Beijupirá Olinda|    Fogão Industrial|    22/01/2024|     Ajuste de chama|
| 23|  Pizzaria Atlântico|               Coifa|    08/03/2024|Substituição de f...|
| 19|      Bodega de Véio|    Forno Industrial|    24/01/2024|Reparo na câma

In [51]:
#excluir df_equipamentos_com_id
del df_equipamentos_com_id

In [52]:
#contar cidades x tipo de estabelecimento
dataset.groupBy('Cidade', 'Tipo' ).count().show()

+--------+-----------+-----+
|  Cidade|       Tipo|count|
+--------+-----------+-----+
| Caruaru| Lavanderia|    2|
|  Recife| Lavanderia|    6|
|  Recife|Restaurante|   14|
|  Olinda| Lavanderia|    3|
|Jaboatão| Lavanderia|    2|
|Paulista| Lavanderia|    2|
|  Olinda|Restaurante|   11|
+--------+-----------+-----+



In [53]:
#renomear colunas
dataset= dataset.withColumnRenamed('Pagamento ', 'Pagamento')

In [54]:
from pyspark.sql.functions import col
#Filtrando todos os pagamentos Pendentes
dataset.filter(col("Pagamento")  == 'Pendente').select('Cliente','Pagamento' ,'Forma').show()

+--------------------+---------+---------+
|             Cliente|Pagamento|    Forma|
+--------------------+---------+---------+
|           Parraxaxá| Pendente|Parcelado|
|             Ponteio| Pendente|Parcelado|
|Bar Restaurante S...| Pendente|Parcelado|
|           Paço Real| Pendente|Parcelado|
|      Bodega de Véio| Pendente|Parcelado|
|  Olinda Art & Grill| Pendente|Parcelado|
|  Pizzaria Atlântico| Pendente|Parcelado|
|  Creperia de Olinda| Pendente|Parcelado|
|              Rápida| Pendente|Parcelado|
|         Santa Clara| Pendente|Parcelado|
|           Nova Vida| Pendente|Parcelado|
|           Primavera| Pendente|Parcelado|
|       Roupas Limpas| Pendente|Parcelado|
|       Brilho do Sol| Pendente|Parcelado|
+--------------------+---------+---------+



In [55]:
# Selecionar colunas específicas
df_selected = dataset.select('Cliente','Tipo','Ordem de Serviço','Pagamento','Forma')
df_selected.show()

+--------------------+-----------+----------------+---------+---------+
|             Cliente|       Tipo|Ordem de Serviço|Pagamento|    Forma|
+--------------------+-----------+----------------+---------+---------+
|               Leite|Restaurante|            2287|     Pago|  Á vista|
|       Chica Pitanga|Restaurante|            2295|     Pago|  Á vista|
|           Parraxaxá|Restaurante|            2301| Pendente|Parcelado|
| Spettus Steak House|Restaurante|            2313|     Pago|  Á vista|
|           Beijupirá|Restaurante|            2292|     Pago|  Á vista|
| Entre Amigos o Bode|Restaurante|            2324|     Pago|  Á vista|
|             Ponteio|Restaurante|            2298| Pendente|Parcelado|
|Capitão Gancho Pi...|Restaurante|            2309|     Pago|  Á vista|
| Restaurante Mustang|Restaurante|            2317|     Pago|  Á vista|
|Bar Restaurante S...|Restaurante|            2305| Pendente|Parcelado|
|Portal do Derby R...|Restaurante|            2321|     Pago|  Á

In [56]:
import pandas as pd
import plotly.express as px

In [57]:
#converter df_equipamentos para utilizar com pandas

df_equipamentos_pandas = df_equipamentos.toPandas()

In [58]:
#Equipamentos diferentes  o cliente 'Chica Pitanga' possui e quais são

df_chica = df_equipamentos_pandas.query("Cliente == 'Chica Pitanga'")
equipamentos_diferentes = df_chica['Equipamento'].unique()
print("Equipamentos do Chica Pitanga:")
for equipamento in equipamentos_diferentes:
  print(equipamento)

Equipamentos do Chica Pitanga:
Liquidificador Industrial
Fogão Industrial
Coifa


In [59]:
#quantos equipamentos foram concertados julho 2024

# Converter a coluna 'Data do Reparo' para um objeto datetime
df_equipamentos_pandas['Data do Reparo'] = pd.to_datetime(df_equipamentos_pandas['Data do Reparo'], format='%d/%m/%Y')

# Filtrar o DataFrame para reparos em julho de 2024
reparos_julho_2024 = df_equipamentos_pandas[df_equipamentos_pandas['Data do Reparo'].dt.month == 7]

# Contar o número de reparos em julho de 2024
num_reparos_julho_2024 = reparos_julho_2024.shape[0]

print(num_reparos_julho_2024)

20


In [60]:
#quais clientes foram feitos reparos em maio

# Filtra o DataFrame para incluir apenas reparos realizados em maio de 2024.
reparo_maio = df_equipamentos_pandas[df_equipamentos_pandas['Data do Reparo'].dt.month == 5]

# Extrai os nomes dos clientes únicos.
clientes_maio = reparo_maio['Cliente'].unique()

# Junta os nomes dos clientes em uma string separada por vírgulas.
lista_clientes = ', '.join(clientes_maio)

# Imprime a lista de clientes.
print(lista_clientes)

Portal do Derby Restaurante, Bodega de Véio, Ponteio, Patuá, Patrácia, Spettus Steak House, Bar Restaurante Santa Cruz, Chica Pitanga, Empório Recife, Primavera, Ponto Limpo, Água Limpa


In [61]:
#contar quantos equipamentos diferentes cada cliente tem

tabela_equipamentos = df_equipamentos_pandas.groupby(['ID', 'Cliente'])['Equipamento'].nunique().reset_index(name='Quantidade de Equipamentos')
print(tabela_equipamentos)


    ID                      Cliente  Quantidade de Equipamentos
0    1                        Leite                           3
1    2                Chica Pitanga                           3
2    4          Spettus Steak House                           3
3    5                    Beijupirá                           3
4    6          Entre Amigos o Bode                           1
5    7                      Ponteio                           2
6    9          Restaurante Mustang                           1
7   10   Bar Restaurante Santa Cruz                           3
8   11  Portal do Derby Restaurante                           2
9   12      Bar e Restaurante União                           2
10  14                   Ponte nova                           3
11  15                   Bode do Nô                           1
12  16             Oficina do Sabor                           3
13  17             Beijupirá Olinda                           4
14  18                     Patrácia     

In [62]:
#Graficos mostrando numero de reparos
import plotly.io as pio
# Agrupar os dados por mês e contar o número de reparos
reparos_por_mes = df_equipamentos_pandas.groupby(df_equipamentos_pandas['Data do Reparo'].dt.month)['Equipamento'].count()

# Criar o gráfico de barras
fig = px.bar(reparos_por_mes, 
             x=reparos_por_mes.index, 
             y=reparos_por_mes.values,
            labels={'x': 'Mês', 'y': 'Número de Reparos'})

# Traduzir os meses para português
meses_pt = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 
           'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
fig.update_layout(xaxis_tickvals=list(range(1, 13)), xaxis_ticktext=meses_pt)

# Exibir o gráfico
fig.show()

In [63]:
#quantas vezes cada cliente chamou em 2024 para fazer reparos

# Agrupar por cliente e contar o número de chamadas
chamadas_por_cliente = df_equipamentos_pandas['Cliente'].value_counts()

# Criar o gráfico de barras usando Plotly Express
fig = px.bar(chamadas_por_cliente, 
             y=chamadas_por_cliente.index, 
             x=chamadas_por_cliente.values,
             labels={'x':'Número de Chamadas', 'y':'Cliente'},
             title='Número de Chamadas por Cliente em 2024')

# Exibir o gráfico
fig.show()

In [64]:
#quantos clientes são do tipo restaurante/lavanderia
df_clientes_pandas = df_clientes.toPandas()
contagem_tipos = df_clientes_pandas['Tipo'].value_counts()
fig = px.pie(contagem_tipos, values=contagem_tipos.values, names=contagem_tipos.index, title='Tipos de Clientes')
fig.show()

In [65]:
#fechar a seção
spark.stop()